In [1]:
import sys
from itertools import count
import numpy as np
import pandas as pd

In [3]:
with open('../data/GPCR_genomic_sequences_Grch37.fasta','r') as fin:
    test_header = fin.readline()
    test_sequence = fin.readline()

In [4]:
def parse_header(header):
    fields = header.split('|')
    return {
        'symbol':fields[0][1:],
        'name':fields[1],
        'id':fields[2],
        'chr':fields[3].split('=')[1],
        'start':int(fields[4].split('=')[1]),
        'end':int(fields[5].split('=')[1]),
        'strand':int(fields[6].split('=')[1])
    }
parse_header(test_header)

{'symbol': 'ACKR1',
 'name': 'atypical chemokine receptor 1 (Duffy blood group)',
 'id': 'ENSG00000213088',
 'chr': '1',
 'start': 159173097,
 'end': 159176290,
 'strand': 1}

In [5]:
def variant_array(sequence):
    base_array = np.frombuffer(test_sequence.encode('ASCII'),dtype='|S1')[:-1][:,np.newaxis]
    seqlength = base_array.shape[0]
    condition_list = [
        base_array==b'A',
        base_array==b'C',
        base_array==b'G',
        base_array==b'T'
    ]
    choice_list = [np.tile(x,(seqlength,1)) \
        for x in [
            np.array((b'C',b'G',b'T')),
            np.array((b'A',b'G',b'T')),
            np.array((b'A',b'C',b'T')),
            np.array((b'A',b'C',b'G'))
        ]
    ]
    return (base_array, np.select(condition_list,choice_list))

test_sequence_array, test_variants_array = variant_array(test_sequence)
np.sum(test_sequence_array==test_variants_array)

0

In [7]:
def synthetic_variants(header, sequence):
    # Parse header
    header_dict_ = parse_header(header)
    # Generate array of alternate bases
    sequence_array_, variant_array_ = variant_array(sequence)
    # Generate array of positions
    pos_array_ = np.arange(len(sequence_array_))
    # Tidy to one variant per row
    pos = np.repeat(pos_array_,3)
    ref = np.repeat(sequence_array_,3).astype(str)
    alt = variant_array_.flatten().astype(str)
    refalt = np.char.add(np.char.add(ref, '/'), alt)
    
    # Calculate absolute (assembly) position 
    if header_dict_['strand'] == 1:
        abs_pos = pos + header_dict_['start']
    elif header_dict_['strand'] == -1:
        abs_pos = - pos + header_dict_['end']
    else:
        print('Error: Strand unrecognised!')
    # Generate ids
    symbol = np.char.add(
        np.char.add(
            header_dict_['symbol'] + ':c.',
            (pos + 1).astype(str)
        ),
        np.char.add(
            np.char.add('.', ref),
            np.char.add('>', alt)
        )
    )
    
    # Sanitise for Ensembl VEP
    output = pd.DataFrame(np.stack((abs_pos,abs_pos,refalt,symbol),axis=1),
                          columns = ['pos_start','pos_end','ref/alt','symbol'])
    # Add chromosome
    output['chr'] = header_dict_['chr']
    # Add strand
    output['strand'] = header_dict_['strand']
    # Reorder columns
    output = output[['chr','pos_start','pos_end','ref/alt','strand','symbol']]
    return output
    
synthetic_variants(test_header,test_sequence)

,chr,pos_start,pos_end,ref/alt,strand,symbol
0,1,159173097,159173097,C/A,1,ACKR1:c.1.C>A
1,1,159173097,159173097,C/G,1,ACKR1:c.1.C>G
2,1,159173097,159173097,C/T,1,ACKR1:c.1.C>T
3,1,159173098,159173098,A/C,1,ACKR1:c.2.A>C
4,1,159173098,159173098,A/G,1,ACKR1:c.2.A>G
...,...,...,...,...,...,...
9577,1,159176289,159176289,A/G,1,ACKR1:c.3193.A>G
9578,1,159176289,159176289,A/T,1,ACKR1:c.3193.A>T
9579,1,159176290,159176290,A/C,1,ACKR1:c.3194.A>C
9580,1,159176290,159176290,A/G,1,ACKR1:c.3194.A>G


In [10]:
with open('../data/GPCR_synthetic_variants.txt','w') as fid:
    print('Synthetic variant file wiped.')
with open('../data/GPCR_genomic_sequences_Grch37.fasta','r') as fin:
    while True:
        header = fin.readline()
        sequence = fin.readline()
        if not sequence: 
            break
        syn_var_ = synthetic_variants(header, sequence)
        syn_var_.to_csv('../data/GPCR_synthetic_variants.txt',sep='\t',mode='a',index=False,header=False)
        

Synthetic variant file wiped.
